# Solving Brio Labyrinth with a Neural Network
*Written by Tobias ?, Vinojan ?, Avi SZYCHTER*

This notebook describes the neural network that we created in order to solve the brio labyrinth situation.

# Dependencies

In [1]:
# Keras
from keras.models import Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, LeakyReLU
from keras import backend as K
from keras.optimizers import Adam
from keras.initializers import Constant

# Other dependencies
import numpy as np
import matplotlib.pyplot as plt
import time
from net_utils import plot_result, load_data_list, visLayer
from preprocess import augmentData, crop



Using TensorFlow backend.


Dectecting IPython...
Matplotlib is configured to use  module://ipykernel.pylab.backend_inline


# Loading and generating data

In [2]:
# -- Load the data
path = 'Annotated_Datasets/' # Configure to fit your file structure 

fileList = ['Annotated_Datasets/whole_roomANDwindowNoonlight.txt',
           'Annotated_Datasets/whole_roomOnNoonWhiteGB.txt',
           'Annotated_Datasets/whole_white_fromCenter.txt',
           'Annotated_Datasets/whole_white_fromNorthEast.txt',
           'Annotated_Datasets/whole_white_fromNorthWest.txt',
           'Annotated_Datasets/whole_white_fromSouthWest.txt',
           'Annotated_Datasets/whole_ledblue.txt',
           'Annotated_Datasets/whole_ledgreen.txt']

imgs_raw, anno_raw = load_data_list(path, fileList)

In [37]:
# -- Augment the data by flipping and adjusting brightness. Also shuffle the data

# -- Values for cropping out the board
xmin = 180
xmax = 650
ymin = 50
ymax = 450

# -- Maximum value for random brightness adjust
brightnessFactor = 20

# -- Augment the data
imgs_all, anno_all = augmentData(imgs_raw, anno_raw, xmin, xmax, ymin, ymax, brightnessFactor)
print(imgs_aug.shape)

(120, 400, 470, 1)


In [ ]:
# -- Split the data in training and validation
splitTrainVal = round(imgs_all.shape[0]*0.80)

x_train_no_norm = imgs_all[:splitTrainVal]
x_val_no_norm = imgs_all[splitTrainVal:]

y_train_no_norm = anno_all[:splitTrainVal]
y_val_no_norm = anno_all[splitTrainVal:]

print(x_train_no_norm.shape)
print(x_val_no_norm.shape)

In [40]:
# -- Normalize the training data
x_mean = [112.60665356606584]
x_std = [53.08789144593608]
print('x_mean: ',x_mean)
print('x_std: ',x_std)

x_train = imgs_all - x_mean
x_train /= x_std

print('x_train dimensions: ',x_train.shape)
print('New mean of x: ',[np.mean(x_train[:,:,:,0])])
print('New std of x: ',[np.std(x_train[:,:,:,0])])


x_mean:  [112.60665356606584]
x_std:  [53.08789144593608]
x_train dimensions:  (120, 100, 100, 1)
New mean of x:  [-0.52339815900651665]
New std of x:  [0.70839702923776471]


In [ ]:
# -- Normalize validation data
x_val = x_val_no_norm.astype('float64') - x_mean
x_val /= x_std

In [ ]:
# -- Scale annotations
y_train = y_train_no_norm / [xmax-xmin,ymax-ymin]
y_val = y_val_no_norm / [xmax-xmin,ymax-ymin]

# Defining the neural network

In [ ]:
# -- Initiliaze the tensorflow session and model
model = None

# -- Proposed to be necessary for running keras in jupyter
session = K.get_session()
if model is not None:
    model.reset_states()

In [ ]:
# -- Dropout and leaky relu parameter
initRelu = Constant(value=0.1)
drop = 0.05

# -- Defining the input layer
x_input = Input(shape=(200, 235, 1))

# -- Defining the convolutional layers
x = Conv2D(15, (3, 3), padding='same', activation=None, bias_initializer=initRelu)(x_input)
x = LeakyReLU(alpha=0.1)(x)
x = Conv2D(15, (3, 3), padding='same', activation=None, bias_initializer=initRelu)(x)

x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop)(x)

x = Conv2D(10, (3, 3), padding='same', activation=None, bias_initializer=initRelu)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Conv2D(10, (3, 3), padding='same', activation=None, bias_initializer=initRelu)(x)

x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop)(x)

x = Conv2D(8, (3, 3), padding='same', activation=None, bias_initializer=initRelu)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dropout(drop)(x)

# -- Flatten output from convolutional layers and add regression head of 4 fully connected layers
x = Flatten()(x)
x = Dense(120)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(120)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(120)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(120)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dropout(drop)(x)

# -- Define output layer 
y_out = Dense(2, activation='linear')(x)

# -- Put the layers together as a model
model = Model(inputs = x_input,outputs = y_out)

print(model.summary()

# Training the neural network

In [ ]:
# -- Define the optimiser for the training
optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# -- Compile the model
model.compile(optimizer=optim, loss='mse')

# -- Train the model
train_history = model.fit(x_train,y_train, epochs=500, validation_data=(x_val,y_val))

In [ ]:
# -- Show the training and validation loss
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.show()

In [ ]:
model.save('Model_big.h5')

# Evaluating the model

In [ ]:
# -- Load data for evaluation (annotated more precise)
imgs_for_test, anno_for_test = load_data_list(path, ['Annotated_Datasets/whole_ledred.txt'])

# -- Augment the data by flipping and adjusting brightness. Also shuffle the data
xmin = 180
xmax = 650
ymin = 50
ymax = 450
brightnessFactor = 20

imgs_aug_t, anno_aug_t = augmentData(imgs_for_test, anno_for_test, xmin, xmax, ymin, ymax, brightnessFactor, resize = (200,235))

# -- Normalize the test data
imgs_test_norm = imgs_aug_t.astype('float64') - x_mean
imgs_test_norm /= x_std

# -- Make predictions
predicts = model.predict_on_batch(imgs_test_norm)

In [75]:
# -- Test the time it takes to make a prediction
sample = imgs_test_norm[0]
tick1 = time.clock()
model.predict(sample)
tick2 = time.clock()

print(tick2-tick1)

0.014525151602356345


In [ ]:
# -- Print the predicted and actual positions and calculate the standard deviation in pixels
losses = []

for i in range(len(x_test)):
    print(i, ' Actual pos:  ', y_test[i]*[xmax-xmin,ymax-ymin])
    print(i, ' Predicted pos', predicts[i]*[xmax-xmin,ymax-ymin])
    print(' ')
    lossX2 = (y_test[i,0]*(xmax-xmin)-predicts[i,0]*(xmax-xmin))**2
    lossY2 = (y_test[i,1]*(ymax-ymin)-predicts[i,1]*(ymax-ymin))**2
    losses.append(lossX2 + lossY2)
    
std = (np.sum(losses)/(len(losses)-1))**(0.5)

In [ ]:
# -- Print the standard deviation
print(std)

In [ ]:
# -- Print the highest error in pixels
print((np.amax(losses))**(0.5))

In [ ]:
# -- Visualise convolutional layers
layer = 16

model2 = Model(inputs = model.input, outputs = model.layers[layer].output)
img = model2.predict(x_test)

print(img.shape)

for i in range(5):

    fig = plt.figure()

    ax = fig.add_subplot(111, title = model2.layers[layer].name + ' filter:' + str(i))
    ax.imshow(img[1,:,:,i], cmap='gray')

In [ ]:
# -- Visual validation of model showing the worst prediction
image = np.argmax(losses)

fig = plt.figure()
    
ax = fig.add_subplot(111)
ax.imshow(imgs_aug_t[image,:,:,0].astype('uint8'), cmap= 'gray')
ax.scatter(predicts[image,0]*235, 
           predicts[image,1]*200, 
           c="r")